# TensorFlow crash course
### **PART 3**

### MNIST Dataset
#### A dataset of images about digits containing 10 classes from 0 to 9

In [1]:
import numpy as np 
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.utils import to_categorical 

(X_train, y_train), (X_test, y_test) = mnist.load_data() 
#  Using less data to speed up training because the main concept isn't about data management
X_train, X_test = X_train[:10000], X_test[:1000]
y_train, y_test = y_train[:10000], y_test[:1000]
X_train, X_test = X_train.reshape(-1, 28*28) / 255.0, X_test.reshape(-1, 28*28) / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(10000, 784) (1000, 784) (10000, 10) (1000, 10)


### Simple DNN(Deep Neural Network) in keras

In [2]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, load_model

dnn = Sequential()
dnn.add(Dense(128, input_shape=(784, ), activation="relu"))
dnn.add(Dense(64, activation="relu"))
dnn.add(Dense(32, activation="relu"))
dnn.add(Dense(10, activation="softmax"))
dnn.compile(loss="categorical_crossentropy", optimizer=SGD(learning_rate=0.01), metrics=["accuracy"])
dnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
dnn.evaluate(X_test, y_test, batch_size=32)
dnn.save("custom_component_model.h5")
dnn = load_model("custom_component_model.h5")
dnn.evaluate(X_test, y_test, batch_size=32)

Epoch 1/10
313/313 [==============================] - 2s 3ms/step - loss: 1.8058 - accuracy: 0.4708 - val_loss: 1.2073 - val_accuracy: 0.6910
Epoch 2/10
313/313 [==============================] - 1s 3ms/step - loss: 0.7502 - accuracy: 0.8106 - val_loss: 0.6073 - val_accuracy: 0.8400
Epoch 3/10
313/313 [==============================] - 1s 3ms/step - loss: 0.4687 - accuracy: 0.8746 - val_loss: 0.4760 - val_accuracy: 0.8570
Epoch 4/10
313/313 [==============================] - 1s 3ms/step - loss: 0.3831 - accuracy: 0.8930 - val_loss: 0.4150 - val_accuracy: 0.8810
Epoch 5/10
313/313 [==============================] - 1s 3ms/step - loss: 0.3356 - accuracy: 0.9054 - val_loss: 0.3898 - val_accuracy: 0.8860
Epoch 6/10
313/313 [==============================] - 1s 3ms/step - loss: 0.3049 - accuracy: 0.9142 - val_loss: 0.3783 - val_accuracy: 0.8840
Epoch 7/10
313/313 [==============================] - 1s 3ms/step - loss: 0.2844 - accuracy: 0.9196 - val_loss: 0.3592 - val_accuracy: 0.8900
Epoch 

[0.30523860454559326, 0.9150000214576721]

### Custom loss function

In [3]:
#  Implement the log loss function with optional <My loss function choice>

import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, load_model

class LogLoss(Loss):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    #  The main formula of the loss function must be written here
    #  If your loss function contains any hyperparameters, initialize it in __init__
    def call(self, y_true, y_pred):
        loss = -tf.reduce_sum(y_true * tf.math.log(y_pred))
        return loss

    #  This function is supposed to map all hyperparameters (including yours) as a dict
    #  When saving the model this function helps save your hyperparameters in a json file
    def get_config(self):
        base_config = super().get_config()
        return {**base_config}

dnn = Sequential()
dnn.add(Dense(128, input_shape=(784, ), activation="relu"))
dnn.add(Dense(64, activation="relu"))
dnn.add(Dense(32, activation="relu"))
dnn.add(Dense(10, activation="softmax"))
dnn.compile(loss=LogLoss(), optimizer=SGD(learning_rate=0.01), metrics=["accuracy"])
dnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
dnn.save("custom_component_model.h5")
#  Use the custom_objects argument to load the loss function along its hyperparameters
dnn = load_model("custom_component_model.h5", custom_objects={"LogLoss":LogLoss})
dnn.evaluate(X_test, y_test, batch_size=32)

Epoch 1/10
313/313 [==============================] - 1s 3ms/step - loss: 20.5487 - accuracy: 0.7977 - val_loss: 11.5133 - val_accuracy: 0.8850
Epoch 2/10
313/313 [==============================] - 1s 2ms/step - loss: 7.6653 - accuracy: 0.9290 - val_loss: 8.0523 - val_accuracy: 0.9160
Epoch 3/10
313/313 [==============================] - 1s 2ms/step - loss: 5.2680 - accuracy: 0.9515 - val_loss: 6.7493 - val_accuracy: 0.9370
Epoch 4/10
313/313 [==============================] - 1s 2ms/step - loss: 6.0429 - accuracy: 0.9492 - val_loss: 7.1796 - val_accuracy: 0.9350
Epoch 5/10
313/313 [==============================] - 1s 2ms/step - loss: 3.5311 - accuracy: 0.9663 - val_loss: 5.8483 - val_accuracy: 0.9500
Epoch 6/10
313/313 [==============================] - 1s 2ms/step - loss: 2.5812 - accuracy: 0.9751 - val_loss: 6.9002 - val_accuracy: 0.9290
Epoch 7/10
313/313 [==============================] - 1s 2ms/step - loss: 2.0221 - accuracy: 0.9802 - val_loss: 7.4061 - val_accuracy: 0.9440
Epoc

[7.48069429397583, 0.9340000152587891]

### Custom layer components
### *Activation function*
### *Regularizer*
### *Initializer*

In [4]:
#  Every structure is going to be like the previous LogLoss structure
#  Activation function : mish
#  Regularizer : L2 
#  Initializer : LeCun

import tensorflow as tf 
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.initializers import Initializer
from tensorflow.keras.models import Sequential, load_model


class Mish(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def call(self, z):
        return tf.multiply(z, tf.math.tanh(self.softplus(z)))

    def softplus(self, z):
        return tf.math.log(tf.exp(z) + 1.0)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config}

class L2(Regularizer):
    def __init__(self, reg=0.0002):
        self.reg = reg
    
    def __call__(self, weights):
        return tf.reduce_sum(tf.multiply(self.reg, weights) ** 2)

    def get_config(self):
        return {"reg":self.reg}

class LeCun(Initializer):
    def __call__(self, shape, dtype=tf.float32):
        limit = tf.sqrt(3 / float(shape[0]))
        return tf.random.normal(shape, stddev=limit, dtype=dtype)

'''Note : You must implement the call() method for losses, layers (including activa‐
tion functions), and models, or the __call__() method for regularizers, initializers,
and constraints'''

dnn = Sequential()
dnn.add(Dense(128, input_shape=(784, ), activation=Mish(), kernel_regularizer=L2(0.0002), kernel_initializer=LeCun))
dnn.add(Dense(64, activation=Mish(), kernel_regularizer=L2(0.0002)))
dnn.add(Dense(32, activation=Mish(), kernel_regularizer=L2(0.0002)))
dnn.add(Dense(10, activation="softmax"))
dnn.compile(loss=LogLoss(), optimizer=SGD(learning_rate=0.01), metrics=["accuracy"])
dnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
dnn.save("custom_component_model.h5")
dnn = load_model("custom_component_model.h5", custom_objects={"LogLoss":LogLoss, "Mish":Mish, "L2":L2, "LeCun":LeCun})
dnn.evaluate(X_test, y_test, batch_size=32)

Epoch 1/10
313/313 [==============================] - 1s 3ms/step - loss: 17.1951 - accuracy: 0.8307 - val_loss: 10.8473 - val_accuracy: 0.8950
Epoch 2/10
313/313 [==============================] - 1s 3ms/step - loss: 6.2465 - accuracy: 0.9390 - val_loss: 7.9605 - val_accuracy: 0.9200
Epoch 3/10
313/313 [==============================] - 1s 3ms/step - loss: 4.2218 - accuracy: 0.9574 - val_loss: 6.1037 - val_accuracy: 0.9360
Epoch 4/10
313/313 [==============================] - 1s 3ms/step - loss: 2.7027 - accuracy: 0.9732 - val_loss: 6.4421 - val_accuracy: 0.9420
Epoch 5/10
313/313 [==============================] - 1s 3ms/step - loss: 2.0159 - accuracy: 0.9802 - val_loss: 7.6799 - val_accuracy: 0.9240
Epoch 6/10
313/313 [==============================] - 1s 3ms/step - loss: 1.7113 - accuracy: 0.9823 - val_loss: 6.7592 - val_accuracy: 0.9380
Epoch 7/10
313/313 [==============================] - 1s 3ms/step - loss: 1.4618 - accuracy: 0.9863 - val_loss: 8.1577 - val_accuracy: 0.9440
Epoc

[7.175950050354004, 0.9449999928474426]